# Переобученная модель
Стоит задача написать модель, которая доходит до хорших метрик, но быстро начинает переобучаться

Импорт необходимых библиотек

In [12]:
import sys

import numpy as np
from PIL import Image
import tensorflow as tf
import torchvision as tv
from livelossplot import PlotLossesKeras

sys.path.append('../../')

from core.datasets import get_ds

Создаем датасеты

In [19]:
# train_ds, val_ds, test_ds = get_ds()
!git pull

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 7 (delta 4), reused 7 (delta 4), pack-reused 0
Unpacking objects: 100% (7/7), 794 bytes | 8.00 KiB/s, done.
From https://github.com/GrishaTS/YaMlProject
   b8d2b7c..9cda5a5  main       -> origin/main
Updating b8d2b7c..9cda5a5
error: Your local changes to the following files would be overwritten by merge:
	models/models/retrained_model.ipynb
Please commit your changes or stash them before you merge.
Aborting


Пишем модель

In [ ]:
model_ex = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.AveragePooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.AveragePooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])
model_ex.summary()

Компиляция модели

In [ ]:
model_ex.compile(optimizer='adam', metrics=['accuracy'],
        loss=tf.keras.losses.SparseCategoricalCrossentropy())

Создаем чекпоинты

In [ ]:
checkpoint_filepath = '../checkpoints/checkpoint_ex.h5'
callback_ = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
)

Обучаем модель

In [ ]:
model_ex.fit(train_ds, validation_data=val_ds, epochs=100,
          callbacks=[PlotLossesKeras(), callback_], verbose=False)

Скачиваем модель из эпохи, которая показала лучшую метрику

In [ ]:
model_ex_best = tf.keras.models.load_model(checkpoint_filepath)
pred = model_ex_best.predict(test_ds / 255., verbose=False)

Формируем ответы в папку ans

In [ ]:
make_ans_file('models/answers/model_ex/checkpoint_ex.ans', pred)